In [ ]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from typing import Callable
import pickle
from ipywidgets import IntProgress
from IPython.display import display

from kaggle_environments import evaluate, make, utils
from kaggle_environments.utils import Struct

# Save model

In [ ]:
class AgentModel(nn.Module):
    
    def __init__(self):
        super(AgentModel, self).__init__()
        self.input_layer = nn.Linear(44, 128)
        self.middle_layer = nn.Linear(128, 128) 
        self.output_layer = nn.Linear(128, 7) 
        self.optimizer = optim.Adam(self.parameters(), lr=0.003)
        self.loss = nn.MSELoss()
        self.device = 'cpu'
        self.to(self.device)

    def load_action(
        self,
        observation_input: list,
        board: list,
        rows: int,
        cols: int
    ) -> int:
        state = T.tensor(observation_input, dtype=T.float32).to('cpu')
        actions = self.forward(state)

        board = np.array(board).reshape(rows, cols).T
        base_actions_list: list = actions.tolist()
        final_actions_list: list = actions.tolist()      
        actions_dict = {k: v for k, v in zip(base_actions_list, range(len(base_actions_list)))}
        
        for i in range(cols):
            if board[i][0]:
                final_actions_list.remove(base_actions_list[i])
        
        if len(final_actions_list):
            action = actions_dict[max(final_actions_list)]
        else:
            action = 0
        
        return action

    def forward(self, state) -> T.tensor:
        x = F.relu(self.input_layer(state))
        x = F.relu(self.middle_layer(x))
        actions = self.output_layer(x)

        return actions

In [ ]:
weights_path = './data/punishment_minus_3_weights'
model = AgentModel()
model.load_state_dict(T.load(weights_path))

In [ ]:
path = './data/model_extra_punishment'

with open(path, 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
loaded_model = None

with open(path, 'rb') as file:
    loaded_model = pickle.load(file)

# Save submission function

In [ ]:
def my_agent(observation: Struct, configuration: Struct) -> int:
    import pickle
    from torch import tensor

    path = 'model_extra_punishment'
    with open(path, 'rb') as file:
        loaded_model = pickle.load(file)

    if configuration is None:
        rows=6
        cols=7
    else:
        rows = configuration.rows
        cols = configuration.cols
        
    observation_input= observation.board + [observation.step, observation.mark]
    action = model.load_action(observation_input, observation.board, rows, cols)

    return action

In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

In [ ]:
write_agent_to_file(my_agent, "./kaggle/working/submission.py")

# Validate submission

In [1]:
# Note: Stdout replacement is a temporary workaround.
import sys

out = sys.stdout
submission = utils.read_file("./kaggle/working/submission.py")
exec(submission)
# agent = utils.get_last_callable(submission)
sys.stdout = out

env = make("connectx", debug=True)
env.run([my_agent, 'random'])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

NameError: name 'utils' is not defined

In [ ]:
exec(submission)